In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.4 MB

In [ ]:
import gradio as gr
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, utils
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from PIL import Image
import os
import numpy as np
import json
import matplotlib.pyplot as plt
from torch.utils.data.dataloader import default_collate

def plot_human_pose(keypoints):
    # Create a figure and axis
    fig, ax = plt.subplots()
    # Plot keypoints
    for i in range(len(keypoints)):
        x, y, _, _ = keypoints[i]
        ax.scatter(x, -y, color='blue')  # Invert y-axis

    # Connect body parts
    connect_lines = [(0, 2), (2, 7),   # Left eye
                     (0, 5), (5, 8),   # Right eye
                     (9,10),  # Left side
                     (11, 12), (12, 24), (11, 23),  # Right side
                     (24,23), (24,26), (23,25),  # Connect ears and wrists
                     (26, 28), (25, 27),
                     (28, 30), (28, 32), (30,32),# Connect left and right pinky fingers
                     (27, 29), (27, 31), (31,29),  # Connect left and right index fingers
                     (12, 14), (11, 13),  # Connect left and right thumbs
                     (14, 16), (13, 15),  # Connect left and right hips
                     (16, 18), (18, 20), (16,20), (16,22),  # Connect left and right knees
                     (15, 17), (15, 19),  # Connect left and right ankles
                     (17, 19), (15, 21)]  # Connect left and right heels
    for line in connect_lines:
        start, end = line
        x_vals = [keypoints[start][0], keypoints[end][0]]
        y_vals = [-keypoints[start][1], -keypoints[end][1]]  # Invert y-axis
        ax.plot(x_vals, y_vals, linewidth=2, color='red')
    ax.set_aspect('equal', adjustable='datalim')

    fig.savefig('1.jpg')
    ax.set_aspect('equal', adjustable='datalim')
    #plt.title('Predicted Pose')
    #plt.axis('off')
    #plt.show()
    img = Image.open('1.jpg').convert('RGB')
    return img





In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        # Assuming the input image size is 256x256, after four pooling layers the image size will be 16x16
        self.fc1 = nn.Linear(2 * 16 * 16, 1000)
        self.fc2 = nn.Linear(1000, 33 * 4)  # Assuming 33 keypoints

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = torch.flatten(x, 1)  # Flatten the tensor for the fully connected layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
model = SimpleCNN()
model.load_state_dict(torch.load('/content/final_model.pt')['model_state_dict'])
model.eval()




SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=512, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=132, bias=True)
)

In [ ]:
def image_classifier(inp):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),  # Resize the images to 256x256
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.RandomHorizontalFlip(),  # Example augmentation
    # Add more augmentations if needed
    ])
    #inp = inp.convert('RGB')
    image_tensor = transform(inp)
    image_tensor = image_tensor.reshape([1,3,32,32])
    output = model(image_tensor)
    test_predict_plot = output.reshape(33,4)
    keypoints = test_predict_plot.cpu().detach().numpy()  # Replace with your 33 key points

    fig, ax = plt.subplots()
    # Plot keypoints
    for i in range(len(keypoints)):
        x, y, _, _ = keypoints[i]
        ax.scatter(x, -y, color='blue')  # Invert y-axis

    # Connect body parts
    connect_lines = [(0, 2), (2, 7),   # Left eye
                     (0, 5), (5, 8),   # Right eye
                     (9,10),  # Left side
                     (11, 12), (12, 24), (11, 23),  # Right side
                     (24,23), (24,26), (23,25),  # Connect ears and wrists
                     (26, 28), (25, 27),
                     (28, 30), (28, 32), (30,32),# Connect left and right pinky fingers
                     (27, 29), (27, 31), (31,29),  # Connect left and right index fingers
                     (12, 14), (11, 13),  # Connect left and right thumbs
                     (14, 16), (13, 15),  # Connect left and right hips
                     (16, 18), (18, 20), (16,20), (16,22),  # Connect left and right knees
                     (15, 17), (15, 19),  # Connect left and right ankles
                     (17, 19), (15, 21)]  # Connect left and right heels
    for line in connect_lines:
        start, end = line
        x_vals = [keypoints[start][0], keypoints[end][0]]
        y_vals = [-keypoints[start][1], -keypoints[end][1]]  # Invert y-axis
        ax.plot(x_vals, y_vals, linewidth=2, color='red')
    ax.set_aspect('equal', adjustable='datalim')

    fig.savefig('1.jpg')
    ax.set_aspect('equal', adjustable='datalim')
    #plt.title('Predicted Pose')
    #plt.axis('off')
    #plt.show()
    img = Image.open('1.jpg').convert('RGB')
    return img





In [ ]:

demo = gr.Interface(
    fn=image_classifier,
    inputs=gr.Image(type="pil"),
    outputs=gr.Image(type="pil"))

demo.launch(debug = True)
# Example usage:


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a3ca6ec6eb40ef2111.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
